## 首先定义一个Pytorch实现的神经网络

In [3]:
# 0.导入若干工具包
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# 1.定义一个简单的网络类
class Net(nn.Module):
    
    def __init__(self):
        """
        初始化函数
        """
        super(Net, self).__init__()
        # 定义第一层卷积神经网络，输入通道维度=1，输出通道维度=6，卷积核大小3*3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 定义第二层卷积神经网络，输入通道维度=6，输出通道维度=16，卷积核大小3*3
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 定义第三层全连接网络
        self.fc1 = nn.Linear(16 * 6 * 6, 120)   # 16*6*6为输入维度，120为最后神经元的维度
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10是因为最后有10分类
    
    def forward(self, x):
        """
        前向传播函数
        :param x: 
        :return: 
        注意：任意卷积层最后要加激活层，池化层
        """
        # 在（2，2）的池化窗口下执行最大池化操作
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2)) # self.conv1()激活层，再经历2*2的池化层
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # 经过卷积层的处理后，张量要进入全连接层，进入先要调整张量的形状
        x = x.view(-1, self.num_flat_features(x)) # 重新将三维的张量设置为二维
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        # 将三维张量的后两维设置为同一个维度
        # 计算size，除了第0个维度上的batch_size
        size = x.size()[1:] # x正常应该是三维的张量，我们取出后两维
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [5]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "D:\study\AI\NLP_Study\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "D:\study\AI\NLP_Study\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "D:\study\AI\NLP_Study\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "D:\study\AI\NLP_Study\Lib\site-packages\tornado

    注意：模型中所有可训练参数，可以通过net.parameters()来获得

In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


    通过params[0].size()所获得的参数是第一层卷积神经网络的参数

In [7]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0951, -0.0597,  0.0496,  0.0312,  0.0073,  0.0755,  0.1126, -0.0279,
          0.1297,  0.1074]], grad_fn=<AddmmBackward0>)


    所获得结果为一行十列，有了输出张量后，就可以执行梯度归零和反向传播的操作

In [8]:
# 反向传播之前：一定要执行梯度清零
net.zero_grad()
out.backward(torch.randn(1, 10))

    注意：
        （1）torch.nn构建的神经网络支支持mini-batches的输入，不支持单一样本的输入
        （2）比如：nn.Conv2d需要一个4DTensor，形状为（nSaples，nChannels，
        Height，Width）。如果你的输入只有单一样本形式，则需要执行input.unsequeeze(0)，
        主动将3DTensor扩充成4DTensor